In [104]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [105]:
df1 = pd.DataFrame({
'고객번호': [1001, 1002, 1003, 1004, 1005, 1006, 1007],
'이름': ['둘리', '도우너', '또치', '길동', '희동', '마이콜', '영희']
}, columns=['고객번호', '이름'])

In [106]:
df2 = pd.DataFrame({
'고객번호': [1001, 1001, 1005, 1006, 1008, 1001],
'금액': [10000, 20000, 15000, 5000, 100000, 30000]
}, columns=['고객번호', '금액'])

In [107]:

df3 = pd.merge(df1, df2)
df3

,고객번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1001,둘리,30000
3,1005,희동,15000
4,1006,마이콜,5000


In [108]:
#Full Outer join
pd.merge(df1, df2, how='outer')

,고객번호,이름,금액
0,1001,둘리,10000.0
1,1001,둘리,20000.0
2,1001,둘리,30000.0
3,1002,도우너,NaN
4,1003,또치,NaN
5,1004,길동,NaN
6,1005,희동,15000.0
7,1006,마이콜,5000.0
8,1007,영희,NaN
9,1008,NaN,100000.0


In [109]:
# Right outer join
pd.merge(df1, df2, how='right')

,고객번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1005,희동,15000
3,1006,마이콜,5000
4,1008,NaN,100000
5,1001,둘리,30000


In [110]:
#left outer join
pd.merge(df1, df2, how='left')

,고객번호,이름,금액
0,1001,둘리,10000.0
1,1001,둘리,20000.0
2,1001,둘리,30000.0
3,1002,도우너,NaN
4,1003,또치,NaN
5,1004,길동,NaN
6,1005,희동,15000.0
7,1006,마이콜,5000.0
8,1007,영희,NaN


In [111]:
df3 = df1.set_index('고객번호')
df4 = df2.set_index('고객번호')
df3.join(df4)

,이름,금액
고객번호,,
1001,둘리,10000.0
1001,둘리,20000.0
1001,둘리,30000.0
1002,도우너,NaN
1003,또치,NaN
1004,길동,NaN
1005,희동,15000.0
1006,마이콜,5000.0
1007,영희,NaN


- 서울시 인구 데이터와 cctv 데이터 합치기

In [112]:
pop = pd.read_csv('서울시_자치구별_주민등록인구.txt', sep = '\t', skiprows=2,encoding='UTF-8')
pop.head()

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2021.3/4,합계,"4,421,143","9,765,869","4,739,972","5,025,897","9,542,256","4,636,069","4,906,187","223,613","103,903","119,710",2.16,"1,592,339"
1,2021.3/4,종로구,"73,652","154,318","74,561","79,757","145,512","70,666","74,846","8,806","3,895","4,911",1.98,"27,776"
2,2021.3/4,중구,"63,267","131,943","64,274","67,669","122,858","59,853","63,005","9,085","4,421","4,664",1.94,"24,225"
3,2021.3/4,용산구,"111,177","238,300","115,655","122,645","224,145","107,801","116,344","14,155","7,854","6,301",2.02,"38,906"
4,2021.3/4,성동구,"134,620","294,140","143,055","151,085","287,820","140,290","147,530","6,320","2,765","3,555",2.14,"46,053"


In [113]:
#1) 필요한 컬럼만 가져오기
pop = pop[['자치구','계','65세이상고령자']]
pop.head()

,자치구,계,65세이상고령자
0,합계,"9,765,869","1,592,339"
1,종로구,"154,318","27,776"
2,중구,"131,943","24,225"
3,용산구,"238,300","38,906"
4,성동구,"294,140","46,053"


In [114]:
#2) 합계 지우기
pop.drop([0], inplace=True)
pop.head(3)

,자치구,계,65세이상고령자
1,종로구,"154,318","27,776"
2,중구,"131,943","24,225"
3,용산구,"238,300","38,906"


In [116]:
# 자치구를 인덱스로, 컬럼명변경
pop.set_index('자치구', inplace=True)
pop.rename(columns={'계':'총인구', '65세이상고령자':'노인인구'}, inplace=True)
pop.head(3)

,총인구,노인인구
자치구,,
종로구,"154,318","27,776"
중구,"131,943","24,225"
용산구,"238,300","38,906"


In [118]:
#4) 숫자를 정수로 변환\
pop = pop.apply(lambda x: x.str.replace(',','')).astype(int)
pop.head(3)

,총인구,노인인구
자치구,,
종로구,154318,27776
중구,131943,24225
용산구,238300,38906


In [119]:
# 3)노인인구비율\
pop['노인인구비율'] = (pop.노인인구 / pop.총인구 *100).round(2) 
pop.head(3)

,총인구,노인인구,노인인구비율
자치구,,,
종로구,154318,27776,18.00
중구,131943,24225,18.36
용산구,238300,38906,16.33


In [120]:
cctv = pd.read_csv('cctv.csv')
cctv.head(3)

,자치구,CCTV댓수,2021년,2017년 이전,최근 3개년,최근 증가율
0,종로구,1772,43,1435,294,20.49
1,중 구,2333,289,1142,902,78.98
2,용산구,2383,0,1334,1049,78.64


In [121]:
cctv.set_index('자치구', inplace=True)
cctv.head(3)

,CCTV댓수,2021년,2017년 이전,최근 3개년,최근 증가율
자치구,,,,,
종로구,1772,43,1435,294,20.49
중 구,2333,289,1142,902,78.98
용산구,2383,0,1334,1049,78.64


In [123]:
df = pop.join(cctv)
df.head()

,총인구,노인인구,노인인구비율,CCTV댓수,2021년,2017년 이전,최근 3개년,최근 증가율
자치구,,,,,,,,
종로구,154318,27776,18.00,1772.0,43.0,1435.0,294.0,20.49
중구,131943,24225,18.36,NaN,NaN,NaN,NaN,NaN
용산구,238300,38906,16.33,2383.0,0.0,1334.0,1049.0,78.64
성동구,294140,46053,15.66,3602.0,76.0,2413.0,1113.0,46.13
광진구,353380,51278,14.51,2588.0,32.0,1204.0,1352.0,112.29


In [125]:
# 인구 1천명당  cctv 댓수
df['1천명당 CCTV댓수'] = (df.CCTV댓수 / df.총인구 * 1000).round(2)
df.tail()

,총인구,노인인구,노인인구비율,CCTV댓수,2021년,2017년 이전,최근 3개년,최근 증가율,1천명당 CCTV댓수
자치구,,,,,,,,,
관악구,501572,79444,15.84,4942.0,34.0,3274.0,1634.0,49.91,9.85
서초구,420045,60493,14.40,3704.0,3.0,2495.0,1206.0,48.34,8.82
강남구,538075,77222,14.35,6502.0,1.0,4022.0,2479.0,61.64,12.08
송파구,664996,96453,14.50,2854.0,101.0,908.0,1845.0,203.19,4.29
강동구,467997,73228,15.65,2547.0,0.0,1200.0,1347.0,112.25,5.44


In [126]:
df.to_csv('서울시자치시구별인구와CCTV')